In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import os
import certifi
import time
import re
import folium
import geopandas as gpd
from folium.features import GeoJsonTooltip
import fiona


# Load the data
df = pd.read_excel("record_24to25.xlsx")
# Drop rows with missing data
df = df.dropna(subset=["patients_common_name", "patients_disposition", "patients_disposition_location"])
released_df = df[df["patients_disposition"] == "Released"].copy()

## Preprocess Address

In [16]:
def clean_address(address: str) -> str:
    if not isinstance(address, str):
        return address

    # Fix common typos
    address = address.replace("PIace", "Place").replace("PI", "Pl")

    # Remove descriptive info in parentheses
    address = re.sub(r'\([^)]*\)', '', address)

    # Handle addresses starting with landmark names
    parts = [p.strip() for p in address.split(',')]
    if len(parts) >= 2 and re.search(r'\d{5}', parts[-1]):
        # Already valid format
        cleaned = ', '.join(parts)
    else:
        # Try to rearrange if ZIP is not at the end
        zip_match = re.search(r'\d{5}', address)
        if zip_match:
            zip_code = zip_match.group()
            before = address[:zip_match.start()]
            after = address[zip_match.start():]
            cleaned = f"{before.strip()}, {after.strip()}"
        else:
            cleaned = address

    return cleaned.strip()

Saved cleaned addresses to cached_geocodes_cleaned.csv


## Get Address Latitude and Longitude

In [15]:
# --- Setup ---
os.environ['SSL_CERT_FILE'] = certifi.where()

# Load released-only addresses
addresses = released_df["patients_disposition_location"].dropna().unique()

# Load existing cache
cache_file = "cached_geocodes.csv"
if os.path.exists(cache_file):
    cached_df = pd.read_csv(cache_file)
    cache = dict(zip(cached_df["address"], zip(cached_df["lat"], cached_df["lon"])))
else:
    cache = {}

# Geocoder setup
geolocator = Nominatim(user_agent="pswc_mapper", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2, max_retries=3, error_wait_seconds=10)

# Process addresses
results = []

for raw_address in tqdm(addresses, desc="Geocoding addresses"):
    cleaned = clean_address(raw_address)
    if cleaned in cache:
        lat, lon = cache[cleaned]
    else:
        try:
            location = geocode(cleaned)
            if location:
                lat, lon = location.latitude, location.longitude
            else:
                lat, lon = None, None
        except Exception:
            lat, lon = None, None
        cache[cleaned] = (lat, lon)
        time.sleep(2)
    results.append({"address": raw_address, "lat": lat, "lon": lon})

# Save cache
geocoded_df = pd.DataFrame(results)
geocoded_df.to_csv(cache_file, index=False)

Geocoding addresses: 100%|██████████| 226/226 [01:40<00:00,  2.26it/s]


## Map Release locations

In [19]:
# Initialize map with the same style as the first map
m = folium.Map(location=[47.6, -122.33], zoom_start=10, tiles="CartoDB positron")

# Add markers
for _, row in geocoded_df.dropna(subset=["lat", "lon"]).iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=row["address"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# Save map
m.save("release_locations_map2.html")

## Map Release Area

In [25]:
# --- Load list of valid municipalities ---
municipalities = pd.read_csv("wa_municipalities_cleaned.csv")  # should have a column 'City'
municipalities["Municipality"] = municipalities["Municipality"].str.lower().str.strip()

# --- Extract city from location field ---
def extract_city(location, city_list):
    location = str(location).lower()
    for city in city_list:
        if city in location:
            return city
    return None

released_df["disposition_city"] = released_df["patients_disposition_location"].apply(
    lambda x: extract_city(x, municipalities["Municipality"].tolist())
)

# --- Count number of releases per city ---
city_counts = released_df["disposition_city"].value_counts().reset_index()
city_counts.columns = ["CityName", "count"]

# --- Load GDB of city boundaries ---
gdb_path = "CityLimits/CityLimits.gdb"
layer_name = fiona.listlayers(gdb_path)[0]
gdf = gpd.read_file(gdb_path, layer=layer_name)
gdf['CityName'] = gdf['CityName'].str.lower().str.strip()

# --- Merge counts with city geometries ---
merged = gdf.merge(city_counts, on="CityName", how="left").fillna({'count': 0})
nonzero = merged[merged['count'] > 0].copy()
nonzero = nonzero.to_crs(epsg=4326)
nonzero = nonzero.drop(columns=["LastUpdate"], errors="ignore")

# --- Plot interactive map ---
m = folium.Map(location=[47.6062, -122.3321], zoom_start=9, tiles="CartoDB positron")

# Add choropleth
folium.Choropleth(
    geo_data=nonzero,
    data=nonzero,
    columns=["CityName", "count"],
    key_on="feature.properties.CityName",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Number of Releases",
    nan_fill_color="white",
).add_to(m)

# Optional: Add tooltip
folium.GeoJson(
    nonzero,
    tooltip=GeoJsonTooltip(
        fields=["CityName", "count"],
        aliases=["City", "Releases"],
        localize=True,
        sticky=True,
        labels=True
    ),
    style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.3},
).add_to(m)

# Save
m.save("release_municipalities_map.html")